In [ ]:
import pandas as pd

# Lista de ciudades y estados de México 
capitales_mexico = {
    "Villahermosa": "Tabasco",
    "Aguascalientes": "Aguascalientes",
    "Mexicali": "Baja California",
    "La Paz": "Baja California Sur",
    "Campeche": "Campeche",
    "Saltillo": "Coahuila",
    "Colima": "Colima",
    "Tuxtla Gutiérrez": "Chiapas",
    "Chihuahua": "Chihuahua",
    "Cd. De México (Zócalo)": "Distrito Federal",
    "Durango": "Durango",
    "Guanajuato": "Guanajuato",
    "Chilpancingo": "Guerrero",
    "Pachuca": "Hidalgo",
    "Guadalajara": "Jalisco",
    "Toluca": "México",
    "Morelia": "Michoacán",
    "Cuernavaca": "Morelos",
    "Tepic": "Nayarit",
    "Monterrey": "Nuevo León",
    "Oaxaca": "Oaxaca",
    "Puebla": "Puebla",
    "Querétaro": "Querétaro",
    "Chetumal": "Quintana Roo",
    "San Luis Potosí": "San Luis Potosí",
    "Culiacán": "Sinaloa",
    "Hermosillo": "Sonora",
    "Cd. Victoria": "Tamaulipas",
    "Tlaxcala": "Tlaxcala",
    "Xalapa": "Veracruz",
    "Mérida": "Yucatán",
    "Zacatecas": "Zacatecas"
}

# Crear un DataFrame a partir del diccionario capitales_mexico
df_ciudades = pd.DataFrame(list(capitales_mexico.items()), columns=["ciudad", "estado"])

# Generar combinaciones sin repeticiones (sin invertir el origen y destino)
combinaciones = [
    (origen["ciudad"], origen["estado"], destino["ciudad"], destino["estado"])
    for i, origen in df_ciudades.iterrows()
    for j, destino in df_ciudades.iterrows()
    if i < j  # Asegura que solo se generen combinaciones sin repeticiones
]

# Crear el DataFrame de combinaciones
df_combinaciones = pd.DataFrame(
    combinaciones,
    columns=["ciudad_origen", "estado_origen", "ciudad_destino", "estado_destino"]
)





# Mostrar el DataFrame completo
print(df_combinaciones)

# Guardar el DataFrame si es necesario
df_combinaciones.to_csv("C:/Users/57323/Documents/combinaciones_ciudades_estados1.csv", index=False) 


    ciudad_origen estado_origen  ciudad_destino       estado_destino
0    Villahermosa       Tabasco  Aguascalientes       Aguascalientes
1    Villahermosa       Tabasco        Mexicali      Baja California
2    Villahermosa       Tabasco          La Paz  Baja California Sur
3    Villahermosa       Tabasco        Campeche             Campeche
4    Villahermosa       Tabasco        Saltillo             Coahuila
..            ...           ...             ...                  ...
491      Tlaxcala      Tlaxcala          Mérida              Yucatán
492      Tlaxcala      Tlaxcala       Zacatecas            Zacatecas
493        Xalapa      Veracruz          Mérida              Yucatán
494        Xalapa      Veracruz       Zacatecas            Zacatecas
495        Mérida       Yucatán       Zacatecas            Zacatecas

[496 rows x 4 columns]


In [ ]:
## web scrapping


import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

# Configuración del driver (asegúrate de tener el ChromeDriver instalado)
driver = webdriver.Chrome()

# Acceder a la página
url = "https://app.sct.gob.mx/sibuac_internet/ControllerUI?action=cmdEscogeRuta"
driver.get(url)

# Esperar a que la página cargue
time.sleep(2)
combinaciones = df_combinaciones

# Lista donde guardaremos los datos
resultados = []

for _, row in combinaciones.iterrows():  
    ciudad_origen = row['ciudad_origen']
    ciudad_destino = row['ciudad_destino']
    estado_origen = row['estado_origen']  # Obteniendo directamente el estado de origen
    estado_destino = row['estado_destino']
    
   

    # Verificar que ambos estados estén definidos
    if estado_origen is None or estado_destino is None:
        print(f"Error: No se encontró el estado para la combinación {ciudad_origen} - {ciudad_destino}")
        continue  # Saltar a la siguiente combinación si no se encontró un estado

    # Mostrar los estados seleccionados para depuración
    print(f"Estado Origen: {estado_origen}, Estado Destino: {estado_destino}")

    # Selecciona el campo de Estado Origen
    estado_origen_element = Select(driver.find_element(By.NAME, 'edoOrigen'))
    estado_origen_element.select_by_visible_text(estado_origen)  # Selecciona el estado de origen

    # Selecciona el campo de Ciudad Origen
    ciudad_origen_element = Select(driver.find_element(By.NAME, 'ciudadOrigen'))
    ciudad_origen_element.select_by_visible_text(ciudad_origen)  # Selecciona la ciudad de origen

    # Selecciona el campo de Estado Destino
    estado_destino_element = Select(driver.find_element(By.NAME, 'edoDestino'))
    try:
        estado_destino_element.select_by_visible_text(estado_destino)  # Selecciona el estado de destino
    except Exception as e:
        print(f"Error al seleccionar el estado de destino: {estado_destino} - {e}")
        continue  # Saltar a la siguiente combinación si ocurre un error

    # Selecciona el campo de Ciudad Destino
    ciudad_destino_element = Select(driver.find_element(By.NAME, 'ciudadDestino'))
    ciudad_destino_element.select_by_visible_text(ciudad_destino)  # Selecciona la ciudad de destino

    # Selecciona el tipo de Vehículo (fijo en este caso)
    vehiculo = Select(driver.find_element(By.NAME, 'vehiculos'))
    vehiculo.select_by_visible_text('Automóvil')

    # Espera un poco antes de pasar al siguiente par de ciudad y estado
    time.sleep(2)

    # Clic en el botón Consultar
    consultar_button = driver.find_element(By.XPATH, "//input[@value='Consultar']")
    consultar_button.click()

    # Espera para que los resultados de la consulta se carguen
    time.sleep(3)

    # Hacer scroll hasta la sección de los totales
    try:
        # Esperar a que la tabla esté presente
        wait = WebDriverWait(driver, 10)
        table = wait.until(EC.presence_of_element_located((By.ID, "tContenido")))

        # Buscar la fila específica con clase "tr_gris"
        rows = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "tr_gris")))

        # Almacenar datos encontrados
        for row in rows:
            # Buscar las celdas <td> dentro de la fila con clase "texto3" que contengan "Totales"
            tds = row.find_elements(By.XPATH, './/td[@class="texto3"]')
            for td in tds:
                if td.text.strip() == 'Totales':
                    # Extraer todas las celdas <td> dentro de la fila
                    cells = row.find_elements(By.TAG_NAME, "td")

                    # Obtener los valores de texto de cada celda
                    longitud = cells[1].text.strip()  # Longitud
                    tiempo = cells[2].text.strip()  # Tiempo
                    automovil = cells[4].text.strip()  # Automóvil
                    
                    # Crear la ruta como una concatenación de ciudad_origen y ciudad_destino
                    ruta = f"{ciudad_origen} - {ciudad_destino}"
                    
                    # Guardar en la lista de resultados con la ruta
                    resultados.append([ruta, longitud, tiempo, automovil])

        # Hacer clic en el botón "Otra Consulta" (suponiendo que existe este botón)
        otra_consulta_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//input[@value='Otra Consulta']"))
        )
        otra_consulta_button.click()

        # Esperar a que la página recargue completamente antes de continuar con la siguiente combinación
        time.sleep(3)

    except Exception as e:
        print(f"Error al procesar la combinación {ciudad_origen} - {ciudad_destino}: {e}")
        continue  # En caso de error, pasar a la siguiente combinación

# Cerrar el navegador después de terminar todas las consultas
driver.quit()

# Crear un DataFrame con los resultados
df_resultados = pd.DataFrame(resultados, columns=["ruta", "longitud", "tiempo", "automovil"])

# Mostrar el DataFrame de resultados
print(df_resultados)


Estado Origen: Tabasco, Estado Destino: México
Estado Origen: Aguascalientes, Estado Destino: México
Estado Origen: Baja California, Estado Destino: México
Estado Origen: Baja California Sur, Estado Destino: México
Estado Origen: Campeche, Estado Destino: México
Estado Origen: Coahuila, Estado Destino: México
Estado Origen: Colima, Estado Destino: México
Estado Origen: Chiapas, Estado Destino: México
Estado Origen: Chihuahua, Estado Destino: México
Estado Origen: Distrito Federal, Estado Destino: México
Estado Origen: Durango, Estado Destino: México
Estado Origen: Guanajuato, Estado Destino: México
Estado Origen: Guerrero, Estado Destino: México
Estado Origen: Hidalgo, Estado Destino: México
Estado Origen: Jalisco, Estado Destino: México
                               ruta   longitud tiempo automovil
0             Villahermosa - Toluca    817.350  08:14   1,343.0
1           Aguascalientes - Toluca    502.650  04:50     956.0
2                 Mexicali - Toluca  2,509.110  24:43   3,54

In [32]:
print(df_resultados)

                               ruta   longitud tiempo automovil
0             Villahermosa - Toluca    817.350  08:14   1,343.0
1           Aguascalientes - Toluca    502.650  04:50     956.0
2                 Mexicali - Toluca  2,509.110  24:43   3,545.0
3                   La Paz - Toluca  4,143.590  45:10   3,976.0
4                 Campeche - Toluca  1,199.900  12:55   1,612.0
5                 Saltillo - Toluca    818.070  08:28     362.0
6                   Colima - Toluca    655.670  06:23   1,344.0
7         Tuxtla Gutiérrez - Toluca    907.050  09:02   1,533.0
8                Chihuahua - Toluca  1,408.620  14:48   1,175.0
9   Cd. De México (Zócalo) - Toluca     70.900  01:06     105.0
10                 Durango - Toluca    876.370  09:52     232.0
11              Guanajuato - Toluca    357.390  03:41     628.0
12            Chilpancingo - Toluca    298.780  03:15     292.0
13                 Pachuca - Toluca    183.900  01:57     241.0
14             Guadalajara - Toluca    4

In [ ]:
# Guardar el DataFrame si es necesario
df_resultados.to_csv("C:/Users/57323/Documents/uno.csv", index=False) ## por temas de tiempo, se sacaban de a 100 filas del dataframe inicial en elweb scrapping, y los resultados se iban pegando en un excel, que se llamo Book1

In [ ]:

## se lee el excel Book1 que fue donde se pegaron los resultados del web scrappping

import pandas as pd

# Leer solo las primeras 4 columnas del archivo Excel
df = pd.read_excel("C:/Users/57323/Downloads/Book1.xlsx", usecols=[0, 1, 2, 3])  # Índices de las columnas (empiezan en 0)

# Mostrar las primeras filas del DataFrame
print(df.head())


                            ruta  longitud  tiempo en horas  automovil
0  Villahermosa - Aguascalientes   1242.90         11:37:00       2154
1        Villahermosa - Mexicali   3321.16   1 day, 8:13:00       5129
2          Villahermosa - La Paz   4952.14  2 days, 4:38:00       5560
3        Villahermosa - Campeche    382.54         04:40:00        269
4        Villahermosa - Saltillo   1580.13         15:12:00       1898


In [3]:
# 19.7 km/l del cheverolet aveo con eso y la distancia calculamos los litros  que se necesitan
df['num_litros'] = (df.iloc[:, 1] / 19.7) 

# precio del combustible, en México el combistibke https://es.globalpetrolprices.com/Mexico/gasoline_prices/ precio del litro 25.19
df['precio_combustible'] = df['num_litros'] * 25.19

In [4]:
## calcular la hora del vendedor, en promedio un vendedor en México El salario vendedor promedio en México es de $ 65,412 al año o $ 33.54 por hora 
# https://mx.talent.com/

import datetime

def convertir_a_timedelta(valor):
    if pd.isnull(valor):  # Si el valor es nulo
        return pd.Timedelta(0)  # Devuelve un timedelta vacío
    if isinstance(valor, str):  # Si el valor es una cadena de texto
        try:
            # Intentamos convertirlo a timedelta
            return pd.to_timedelta(valor)
        except ValueError:
            return pd.Timedelta(0)  # Si no puede convertirse, devolver 0
    elif isinstance(valor, datetime.time):
        # Si es datetime.time, lo convertimos a timedelta (diferencia desde medianoche)
        return pd.Timedelta(hours=valor.hour, minutes=valor.minute, seconds=valor.second)
    else:
        return valor  # Si ya es timedelta, lo dejamos como está

# Aplicar la conversión a la columna de tiempo (índice 2 en este caso)
df.iloc[:, 2] = df.iloc[:, 2].apply(convertir_a_timedelta)




In [5]:
# Crear la nueva columna 'minutos_totales'
df['horas_totales'] = df.iloc[:, 2].apply(lambda x: x.total_seconds() / 3600)

# Calcular el costo en función del tiempo trabajado
df['costo_horas_trabajadas'] = (df['horas_totales'] ) * 33.54

In [6]:
## costo total

# Sumar las columnas 4, 5 y 6 (índices 3, 4 y 5)
df['costo_total'] = df.iloc[:, 3] + df.iloc[:, 5] + df.iloc[:, 7]

# Mostrar las primeras filas del DataFrame con la nueva columna
print(df.head(400))

df


                              ruta  longitud  tiempo en horas  automovil  \
0    Villahermosa - Aguascalientes   1242.90  0 days 11:37:00       2154   
1          Villahermosa - Mexicali   3321.16  1 days 08:13:00       5129   
2            Villahermosa - La Paz   4952.14  2 days 04:38:00       5560   
3          Villahermosa - Campeche    382.54  0 days 04:40:00        269   
4          Villahermosa - Saltillo   1580.13  0 days 15:12:00       1898   
..                             ...       ...              ...        ...   
395           Monterrey - Tlaxcala    999.22  0 days 09:45:00       1007   
396             Monterrey - Xalapa   1204.67  0 days 12:01:00       1295   
397            Monterrey - MÃ©rida   2173.84  0 days 22:30:00       2267   
398          Monterrey - Zacatecas    439.00  0 days 05:17:00         52   
399                Oaxaca - Puebla    344.02  0 days 03:18:00        365   

     num_litros  precio_combustible  horas_totales  costo_horas_trabajadas  \
0     63.

,ruta,longitud,tiempo en horas,automovil,num_litros,precio_combustible,horas_totales,costo_horas_trabajadas,costo_total
0,Villahermosa - Aguascalientes,1242.90,0 days 11:37:00,2154,63.091371,1589.271624,11.616667,389.623,4132.894624
1,Villahermosa - Mexicali,3321.16,1 days 08:13:00,5129,168.586802,4246.701543,32.216667,1080.547,10456.248543
2,Villahermosa - La Paz,4952.14,2 days 04:38:00,5560,251.377665,6332.203381,52.633333,1765.322,13657.525381
3,Villahermosa - Campeche,382.54,0 days 04:40:00,269,19.418274,489.146325,4.666667,156.520,914.666325
4,Villahermosa - Saltillo,1580.13,0 days 15:12:00,1898,80.209645,2020.480949,15.200000,509.808,4428.288949
...,...,...,...,...,...,...,...,...,...
490,Durango - Toluca,876.37,0 days 09:52:00,232,44.485787,1120.596970,9.866667,330.928,1683.524970
491,Guanajuato - Toluca,357.39,0 days 03:41:00,628,18.141624,456.987518,3.683333,123.539,1208.526518
492,Chilpancingo - Toluca,298.78,0 days 03:15:00,292,15.166497,382.044071,3.250000,109.005,783.049071
493,Pachuca - Toluca,183.90,0 days 01:57:00,241,9.335025,235.149289,1.950000,65.403,541.552289


In [7]:

import pandas as pd
import numpy as np

def crear_matriz_costos(df, col_combinaciones=0, col_costos=8):
    # Separar las combinaciones en dos ciudades
    ciudades_pares = df.iloc[:, col_combinaciones].str.split('-')
    
    # Obtener lista única de ciudades
    todas_ciudades = set()
    for par in ciudades_pares:
        todas_ciudades.add(par[0].strip())
        todas_ciudades.add(par[1].strip())
    todas_ciudades = sorted(list(todas_ciudades))
    
    # Crear matriz vacía
    n_ciudades = len(todas_ciudades)
    matriz_df = pd.DataFrame(0.0, index=todas_ciudades, columns=todas_ciudades)
    
    # Llenar la matriz
    for idx, row in df.iterrows():
        ciudad1, ciudad2 = row.iloc[col_combinaciones].split('-')
        ciudad1, ciudad2 = ciudad1.strip(), ciudad2.strip()
        costo = row.iloc[col_costos]
        
        # Asignar el costo en ambas direcciones
        matriz_df.loc[ciudad1, ciudad2] = costo
        matriz_df.loc[ciudad2, ciudad1] = costo
    
    return matriz_df

# Ejemplo de uso:
"""
# Si tu DataFrame se ve así:
data = {
    1: ['villahermosa-mexicali', 'villahermosa-aguascalientes', 'villahermosa-puebla'],
    7: [100, 150, 200]
}
df = pd.DataFrame(data)

# Crear la matriz
matriz = crear_matriz_costos(df)

# Configurar visualización completa
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Ver la matriz completa
print(matriz)

# Para guardar en Excel
matriz.to_excel("matriz_costos.xlsx")
"""

'\n# Si tu DataFrame se ve así:\ndata = {\n    1: [\'villahermosa-mexicali\', \'villahermosa-aguascalientes\', \'villahermosa-puebla\'],\n    7: [100, 150, 200]\n}\ndf = pd.DataFrame(data)\n\n# Crear la matriz\nmatriz = crear_matriz_costos(df)\n\n# Configurar visualización completa\npd.set_option(\'display.max_rows\', None)\npd.set_option(\'display.max_columns\', None)\npd.set_option(\'display.width\', None)\n\n# Ver la matriz completa\nprint(matriz)\n\n# Para guardar en Excel\nmatriz.to_excel("matriz_costos.xlsx")\n'

In [8]:
matriz = crear_matriz_costos(df)
matriz

,Aguascalientes,Campeche,Cd. De MÃ©xico (ZÃ³calo),Cd. Victoria,Chetumal,Chihuahua,Chilpancingo,Colima,Cuernavaca,CuliacÃ¡n,...,QuerÃ©taro,Saltillo,San Luis PotosÃ­,Tepic,Tlaxcala,Toluca,Tuxtla GutiÃ©rrez,Villahermosa,Xalapa,Zacatecas
Aguascalientes,0.000000,5048.119949,1656.053807,1126.497914,5150.608858,2615.243330,2391.775122,1626.762274,1819.093391,3754.690817,...,987.869264,859.898518,439.153442,1705.341614,2131.204457,1760.838604,4464.424239,4132.894624,2958.383711,250.487350
Campeche,5048.119949,0.000000,3361.613411,3910.680533,656.006254,7123.512655,3898.993736,6180.026832,3349.964208,8307.942589,...,4061.927685,5342.955274,4432.797985,6258.593386,2997.521954,3579.513376,1469.668000,914.666325,2653.426051,5298.594513
Cd. De MÃ©xico (ZÃ³calo),1656.053807,3361.613411,0.000000,1728.076315,3464.102320,3731.446513,873.538944,2793.550690,300.857213,4848.435447,...,669.861543,1950.901919,1040.731843,2799.658030,457.163360,232.552426,2777.917701,2446.388086,1271.889959,1907.087371
Cd. Victoria,1126.497914,3910.680533,1728.076315,0.000000,4013.169442,2871.947680,2463.797629,2589.880086,1891.115898,4712.764843,...,1018.629234,652.647919,615.262518,2663.987426,1609.208665,1469.196310,3326.984822,2996.014208,1378.029949,941.208421
Chetumal,5150.608858,656.006254,3464.102320,4013.169442,0.000000,7226.001563,4000.923645,6287.546741,3452.453117,8410.431497,...,4163.857594,5445.444183,4535.286893,6361.082294,3100.010863,3696.107670,1571.597909,1017.168020,2755.914959,5401.083421
Chihuahua,2615.243330,7123.512655,3731.446513,2871.947680,7226.001563,0.000000,5007.018452,4247.036604,3894.486096,4238.754376,...,3021.440431,2219.299761,2690.155670,4861.138513,4206.597162,3472.566508,6539.816944,6208.287330,5034.335416,2364.209766
Chilpancingo,2391.775122,3898.993736,873.538944,2463.797629,4000.923645,5007.018452,0.000000,1727.510843,557.337569,5431.975716,...,1405.023858,2686.623234,1776.453157,3383.185513,1011.673492,783.049071,3315.298025,2983.768411,1608.794305,2642.249685
Colima,1626.762274,6180.026832,2793.550690,2589.880086,6287.546741,4247.036604,1727.510843,0.000000,2736.145934,3458.136508,...,1799.330492,2807.674690,1902.548401,1408.787305,3268.129553,2396.489249,5601.362122,5265.360508,4095.880594,1882.826838
Cuernavaca,1819.093391,3349.964208,300.857213,1891.115898,3452.453117,3894.486096,557.337569,2736.145934,0.000000,4859.030690,...,832.342127,2113.941503,1203.771426,2809.681487,462.643964,267.141904,2766.268497,2434.738883,1267.911147,2069.567954
CuliacÃ¡n,3754.690817,8307.942589,4848.435447,4712.764843,8410.431497,4238.754376,5431.975716,3458.136508,4859.030690,0.000000,...,3921.462538,3707.962853,4025.433157,2051.424523,5391.014310,4519.374005,7724.246878,7392.717264,6017.743157,2377.320726


In [9]:
print("Usando shape:", matriz.shape) 

matriz.to_csv("C:/Users/57323/Downloads/matriz_costos.csv", index=True)

Usando shape: (32, 32)
